# Language Baseline Models

These language models serve as a baseline and will be leverage the book title and description in order to hopefully enhance the predictive power.

In [1]:
import json
import os
import random
import numpy as np
import pandas as pd

random.seed(42)
np.random.seed(42)

### Load Training Data

In [2]:
OUTPUT_DATA_DIR = "./output_data/"

train_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_training.csv")

### Load Validation Data

In [3]:
val_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_validation.csv")

In [4]:
pd.set_option('display.max_columns', None)

### Sample Data For Prototyping

In [5]:
train_df = train_df.sample(frac=0.25)

### Clean Data For Textual Analysis

In [6]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def process_book_text(book_text, exclude_text, ps):
    """Pre-processes the text given by `review_text`.
    
    Parameters
    ----------
    book_text: str
        The book text to be processed.
    exclude_text: collection
        A collection of words to be excluded.
    ps: PorterStemmer
        The PorterStemmer used to perform word stemming.
    
    Returns
    -------
    str
        A string representing the processed version of `review_text`.
    
    """
    book = re.sub('[^a-zA-Z0-9]', ' ', book_text).lower().split()
    book = [ps.stem(word) for word in book if not word in exclude_text]
    return ' '.join(book)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Matthew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def preprocess_for_classification(data_df):
    """Preprocesses `data_df` to be used in classification.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame to be processed.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after processing.
    
    """
    # flags for most popular formats
    data_df['format'] = data_df['format'].apply(lambda x: str(x).lower())
    data_df['is_paperback'] = data_df['format'].apply(lambda x: int("paper" in x))
    data_df['is_hardcover'] = data_df['format'].apply(lambda x: int("hard" in x))
    data_df['is_audio'] = data_df['format'].apply(lambda x: int("audio" in x))
    data_df['is_other_format'] = (data_df['is_paperback'] + data_df['is_hardcover'] + 
                                  data_df['is_audio'] + data_df['is_ebook'])
    data_df['is_other_format'] = data_df['is_other_format'].apply(lambda x: 0 if x > 0 else 1)
    
    #flags for most popular publishers
    data_df['publisher'] = data_df['publisher'].apply(lambda x: str(x).lower())
    data_df['from_penguin'] = data_df['publisher'].apply(lambda x: int("penguin" in x))
    data_df['from_harpercollins'] = data_df['publisher'].apply(lambda x: int("harpercollins" in x or "harper collins" in x))
    data_df['from_university_press'] = data_df['publisher'].apply(lambda x: int("university press" in x))
    data_df['from_vintage'] = data_df['publisher'].apply(lambda x: int("vintage" in x))
    data_df['from_createspace'] = data_df['publisher'].apply(lambda x: int("createspace" in x or "create space" in x))
    data_df['other_publisher'] = (data_df['from_penguin'] + data_df['from_harpercollins'] + 
                                  data_df['from_university_press'] + data_df['from_vintage'] + data_df['from_createspace'])
    data_df['other_publisher'] = data_df['other_publisher'].apply(lambda x: 0 if x > 0 else 1)
    
    # ensuring columns are not missing
    data_df['average_rating'] = data_df['average_rating'].apply(lambda x: 0.0 if pd.isnull(x) else x)
    data_df['text_reviews_count'] = data_df['text_reviews_count'].apply(lambda x: 0 if pd.isnull(x) else x)
    data_df['ratings_count'] = data_df['ratings_count'].apply(lambda x: 0 if pd.isnull(x) else x)
    
    median_page_count = data_df['num_pages'].median()
    data_df['num_pages'] = data_df['num_pages'].apply(lambda x: median_page_count if pd.isnull(x) else x)
    
    # flags for most popular authors
    data_df['main_author'] = data_df['main_author'].astype(str)
    data_df['author_a'] = data_df['main_author'].apply(lambda x: int(x == "435477.0"))
    data_df['author_b'] = data_df['main_author'].apply(lambda x: int(x == "903.0"))
    data_df['author_c'] = data_df['main_author'].apply(lambda x: int(x == "947.0"))
    data_df['author_d'] = data_df['main_author'].apply(lambda x: int(x == "4624490.0"))
    data_df['author_e'] = data_df['main_author'].apply(lambda x: int(x == "18540.0"))
    data_df['author_f'] = data_df['main_author'].apply(lambda x: int(x == "8075577.0"))
    data_df['author_other'] = (data_df['author_a'] + data_df['author_b'] + 
                                data_df['author_c'] + data_df['author_d'] + 
                                data_df['author_e'] + data_df['author_f'])
    data_df['author_other'] = data_df['author_other'].apply(lambda x: 0 if x > 0 else 1)
    return data_df

In [8]:
def preprocess_all_book_text(data_df, id_col, text_col, exclude_text, ps):
    """Preprocesses the book text in `data_df` for `text_col`.
    
    The dataframe is restricted to `id_col` and `text_col` and then the
    unique ids are chosen. This is so that we only preprocess the text
    for a book once. Then we join the resulting text back to `data_df`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame containing the data to be preprocessed.
    id_col: str
        The column from which unique ids are chosen.
    text_col: str
        The column to be pre-processed.
    exclude_text: collection
        A collection of words to remove
    ps: PorterStemmer
        The PorterStemmer used for word stemming.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after adding a column
        with the processed text.
    
    """
    book_df = data_df[[id_col, text_col]]
    book_df = book_df.drop_duplicates(subset=[id_col])
    book_df['cleaned_text'] = book_df[text_col].apply(lambda x: process_book_text(x, exclude_text, ps))
    print(len(data_df))
    final_df = pd.merge(data_df, book_df[[id_col, "cleaned_text"]], how="inner", on=[id_col])
    return final_df

In [9]:
def run_preprocess_pipeline(data_df, exclude_text, ps):
    """Runs the full pre-processing pipeline on `data_df`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
    
    """
    processed_df = preprocess_for_classification(data_df)
    return preprocess_all_book_text(processed_df, "book_id", "title_description", exclude_text, ps)

In [10]:
format_counts = train_df.groupby(train_df['format'])['user_id'].count()
format_counts.sort_values()

format
Hardcover and Paperback                           1
Paperback, E-book                                 1
Paper, Color Photo and Black Twine                1
Trade paperback                                   1
Turtleback                                        1
cards in envelope                                 1
Letterpressed Chapbook                            1
chapbook/ebook                                    1
Hardcover/Paperback                               1
hand-stitched chapbook                            1
Hardcover first edition                           1
Poetry chapbook                                   1
saddle stitched chapbook                          1
Handmade Chapbook                                 1
Chapbook/Pamphlet                                 1
perfect bound                                     1
print                                             1
Board book                                        1
japanese stab bound                               1
hardb

In [11]:
publisher_counts = train_df.groupby(train_df['publisher'])['user_id'].count()
publisher_counts = publisher_counts.sort_values()
publisher_counts[-20:]

publisher
Mariner Books                     517
Simon & Schuster                  520
City Lights                       528
Graywolf Press                    564
HarperCollins                     566
CreateSpace                       665
Penguin Books                     699
Penguin                           718
Rupa & Co                         743
Everyman's Library                756
New Directions                    802
Oxford University Press           821
Doubleday & Company, Inc.         858
Andrews McMeel Publishing         905
Createspace                       921
Dover Publications                938
Vintage                          1018
Cambridge University Press       2442
Harpercollins Childrens Books    4448
Penguin Classics                 5920
Name: user_id, dtype: int64

In [12]:
print(len(train_df[pd.isnull(train_df['average_rating'])]))
print(len(train_df[pd.isnull(train_df['text_reviews_count'])]))
print(len(train_df[pd.isnull(train_df['num_pages'])]))
print(len(train_df[pd.isnull(train_df['ratings_count'])]))
train_df['num_pages'].median()

0
0
2642
0


203.0

In [13]:
author_counts = train_df.groupby(train_df['main_author'])['user_id'].count()
author_counts = author_counts.sort_values()
print(author_counts.values[-10:])
print(author_counts.index[-6:])

[ 824  862  952  972 1104 1283 1814 3626 4118 4528]
Float64Index([8075577.0, 18540.0, 4624490.0, 947.0, 903.0, 435477.0], dtype='float64', name='main_author')


In [14]:
book_df = train_df[['book_id', 'title_description']]
book_df = book_df.drop_duplicates(subset=['book_id'])
book_df

,book_id,title_description
113914,18295863,Letter Composed During a Lull in the Fighting ...
224359,18003300,Love & Misadventure Lang Leav is a poet and in...
90635,46199,"Letters to a Young Poet In 1903, a student at ..."
113523,30119,Where the Sidewalk Ends Where the Sidewalk End...
196420,5289,Complete Works of Oscar Wilde In print since 1...
...,...,...
38204,35498776,The Sky Threw Stars the storm. the strike. the...
172382,874604,Collected Verse of Edgar A. Guest
173754,31122069,The Lay of Aotrou and Itroun The Lay of Aotrou...
67472,22736736,September First


In [15]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
train_df_processed = run_preprocess_pipeline(train_df, exclude_english, ps)
val_df_processed = run_preprocess_pipeline(val_df, exclude_english, ps)

57208
49036


In [16]:
from sklearn.utils import shuffle

def shuffle_dataset(data_df):
    """Randomly shuffles `df`.
    
    Parameters
    ----------
    df: pd.DataFrame
        The DataFrame to be shuffled.
    
    Returns
    -------
    pd.DataFrame
        A shuffled dataframe obtained from `df`.
    
    """
    data_df = shuffle(data_df)
    data_df.reset_index(inplace=True, drop=True)
    return data_df

We reshuffle the datasets as they were sorted for the merge

In [17]:
train_df_processed = shuffle_dataset(train_df_processed)
val_df_processed = shuffle_dataset(val_df_processed)

In [18]:
train_df_processed.to_csv(OUTPUT_DATA_DIR+"text_processed_training.csv")
val_df_processed.to_csv(OUTPUT_DATA_DIR+"text_processed_validation.csv")

We take a set of non text features which we will add to our textual model

In [19]:
columns_to_keep = ['text_reviews_count', 'is_ebook', 'average_rating', 'num_pages', 
                   'publication_year', 'ratings_count', 'is_translated', 'is_in_series',
                   'series_length', 'is_paperback', 'is_hardcover', 'is_audio', 'is_other_format',
                   'from_penguin', 'from_harpercollins', 'from_university_press', 'from_vintage',
                   'from_createspace', 'other_publisher', 'author_a', 'author_b', 'author_c',
                   'author_d', 'author_e', 'author_f', 'author_other']
X_train_reg = train_df_processed[columns_to_keep]
X_val_reg = val_df_processed[columns_to_keep]

In [20]:
def log_transform_columns(data_df, cols):
    """Applies a log transform to `cols` in `data_df`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame in which the columns will be transformed.
    cols: collection
        The columns in `data_df` to be log scaled.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after log scaling
        the columns `cols`.
    
    """
    for col in cols:
        data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)
    return data_df

In [21]:
log_transform_cols = ['text_reviews_count', 'ratings_count']
X_train_reg = log_transform_columns(X_train_reg, log_transform_cols)
X_val_reg = log_transform_columns(X_val_reg, log_transform_cols)

<ipython-input-20-88fb59fe720e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)


In [22]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_train_reg = min_max_scaler.fit_transform(X_train_reg)
X_val_reg = min_max_scaler.transform(X_val_reg)

### Building the Textual Model

We apply TF-IDF only to the unique corpus of book descriptions. We only want to apply it to the unique descriptions because we do not want to overweight the books that are frequently read.

We start by getting the book corpus.

In [23]:
book_df = train_df_processed[['book_id', 'cleaned_text']]
book_df = book_df.drop_duplicates(subset=['book_id'])

So we fit TF-IDF to the book corpus and then use it to transform the training and validation text to a sparse matrix

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_model = TfidfVectorizer()

tfidf_model.fit(book_df['cleaned_text'])

train_tfidf = tfidf_model.transform(train_df_processed['cleaned_text'])
val_tfidf = tfidf_model.transform(val_df_processed['cleaned_text'])

Now we combine the language features with the remaining features. As the language features are already in the form of a sparse matrix we convert the remaining_features to a sparse matrix as a sparse matrix will be more efficient than a dataframe for training

In [25]:
import scipy.sparse as sp

X_train_reg_sp = sp.csr_matrix(X_train_reg)
X_train_tfidf_reg = sp.hstack((train_tfidf, X_train_reg_sp), format='csr')

X_val_reg_sp = sp.csr_matrix(X_val_reg)
X_val_tfidf_reg = sp.hstack((val_tfidf, X_val_reg_sp), format='csr')

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

learning_rates = [0.1, 0.3]
estimators = [1000, 2000]
depths = [1, 2]

lrs = []
estim = []
ds = []
train_MSEs = []
val_MSEs = []

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            lrs.append(learning_rate)
            estim.append(estimator)
            ds.append(depth)
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("---------------------------------------------------")
            xg_cls = XGBClassifier(
                objective='binary:logistic', learning_rate=learning_rate,
                max_depth=depth, n_estimators=estimator)
            xg_cls.fit(X_train_tfidf_reg, train_df_processed['recommended'])
            train_MSEs.append(roc_auc_score(
                train_df_processed['recommended'], xg_cls.predict(X_train_tfidf_reg)))
            val_MSEs.append(roc_auc_score(
                val_df_processed['recommended'], xg_cls.predict(X_val_tfidf_reg)))
            print("Training AUC: {}".format(train_MSEs[-1]))
            print("Validation AUC: {}".format(val_MSEs[-1]))
            print()

Learning Rate: 0.1, # Estimators: 1000, Depth: 1
---------------------------------------------------
Training AUC: 0.6644482249336027
Validation AUC: 0.6483601488609546

Learning Rate: 0.1, # Estimators: 1000, Depth: 2
---------------------------------------------------
Training AUC: 0.672873184433992
Validation AUC: 0.647195364583714

Learning Rate: 0.1, # Estimators: 2000, Depth: 1
---------------------------------------------------
Training AUC: 0.670517866118814
Validation AUC: 0.6485175352195953

Learning Rate: 0.1, # Estimators: 2000, Depth: 2
---------------------------------------------------
Training AUC: 0.679544731424684
Validation AUC: 0.6460283237657941

Learning Rate: 0.3, # Estimators: 1000, Depth: 1
---------------------------------------------------
Training AUC: 0.6736934733023123
Validation AUC: 0.6470388362988286

Learning Rate: 0.3, # Estimators: 1000, Depth: 2
---------------------------------------------------
Training AUC: 0.6847127781862746
Validation AUC: 0.64

In [27]:
RESULTS_DIR = './results/'

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

In [28]:
xgb_tfidf = pd.DataFrame({'learning_rate': lrs,
                          'n_estimators': estim,
                          'max_depth': ds,
                          'training_MSE': train_MSEs,
                          'validation_MSE': val_MSEs})
xgb_tfidf.to_csv(RESULTS_DIR+"xgbTFIDF.csv", index=False)

In [29]:
xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.1, max_depth=1, n_estimators=2000)

xg_cls.fit(X_train_tfidf_reg, train_df_processed['recommended'])

train_MSE = roc_auc_score(
                train_df_processed['recommended'], xg_cls.predict(X_train_tfidf_reg))
val_MSE = roc_auc_score(
                val_df_processed['recommended'], xg_cls.predict(X_val_tfidf_reg))
print("Training AUC: {}".format(train_MSE))
print("Validation AUC: {}".format(val_MSE))

Training AUC: 0.670517866118814
Validation AUC: 0.6485175352195953


### Word2Vec Model

In this section we prototype a Word2Vec Model

In [30]:
from gensim.models import Word2Vec

In [31]:
book_df = train_df_processed[['book_id', 'cleaned_text']]
book_df = book_df.drop_duplicates(subset=['book_id'])
w2v = Word2Vec(list(book_df['cleaned_text']), size=200, window=10, min_count=1)

In [32]:
def create_book_vector(book_text, vec_length):
    """Creates a vector for the book given by `book_text`.
    
    The word vectors for each word in `book_text` are 
    averaged to build a vector for the book.
    
    Parameters
    ----------
    book_text: str
        The book text for which the vector is generated.
    
    Returns
    -------
    vector
        A vector for the book.
    
    """
    text_vecs = [word for word in book_text if word in w2v.wv.vocab]
    if len(text_vecs) > 0:
        return np.mean(w2v[text_vecs], axis=0)
    return np.zeros(vec_length)

In [33]:
train_df_processed['book_vector'] = train_df_processed['cleaned_text'].apply(lambda x: create_book_vector(x, 200))
val_df_processed['book_vector'] = val_df_processed['cleaned_text'].apply(lambda x: create_book_vector(x, 200))

<ipython-input-32-aab46b39c3fa>:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[text_vecs], axis=0)


In [34]:
def create_book_vec_df(book_vecs, indices):
    """Creates a dataframe from `book_vecs`.
    
    Each numpy array in `book_vecs` is converted to a 
    row in the resulting dataframe.
    
    Parameters
    ----------
    book_vecs: list
        A list of numpy arrays where each array corresponds
        to the book vector for a book.
    indicies: np.array
        A numpy array of indices for the DataFrame
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from converting `review_vecs`
        to a dataframe.
    
    """
    book_vec_df = pd.DataFrame(np.vstack(book_vecs))
    book_vec_df.columns = ["word" + str(col) for col in book_vec_df.columns]
    book_vec_df.index = indices
    return book_vec_df

In [35]:
train_wv = create_book_vec_df(train_df_processed['book_vector'], train_df_processed.index)
val_wv = create_book_vec_df(val_df_processed['book_vector'], val_df_processed.index)

In [36]:
columns_to_keep = ['text_reviews_count', 'is_ebook', 'average_rating', 'num_pages', 
                   'publication_year', 'ratings_count', 'is_translated', 'is_in_series',
                   'series_length', 'is_paperback', 'is_hardcover', 'is_audio', 'is_other_format',
                   'from_penguin', 'from_harpercollins', 'from_university_press', 'from_vintage',
                   'from_createspace', 'other_publisher', 'author_a', 'author_b', 'author_c',
                   'author_d', 'author_e', 'author_f', 'author_other']
X_train_reg = train_df_processed[columns_to_keep]
X_val_reg = val_df_processed[columns_to_keep]

In [37]:
log_transform_cols = ['text_reviews_count', 'ratings_count']
X_train_reg = log_transform_columns(X_train_reg, log_transform_cols)
X_val_reg = log_transform_columns(X_val_reg, log_transform_cols)

<ipython-input-20-88fb59fe720e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)


In [38]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_train_reg = min_max_scaler.fit_transform(X_train_reg)
X_val_reg = min_max_scaler.transform(X_val_reg)

In [39]:
X_train_reg_df = pd.DataFrame(np.vstack(X_train_reg))
X_train_reg_df.index = train_df_processed.index

X_val_reg_df = pd.DataFrame(np.vstack(X_val_reg))
X_val_reg_df.index = val_df_processed.index

In [40]:
X_train_wv_reg = sp.csr_matrix(pd.concat([train_wv, X_train_reg_df], axis=1))
X_val_wv_reg = sp.csr_matrix(pd.concat([val_wv, X_val_reg_df], axis=1))

In [41]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

learning_rates = [0.1, 0.3]
estimators = [1000, 2000]
depths = [1, 2]

lrs = []
estim = []
ds = []
train_MSEs = []
val_MSEs = []

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            lrs.append(learning_rate)
            estim.append(estimator)
            ds.append(depth)
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("---------------------------------------------------")
            xg_cls = XGBClassifier(
                objective='binary:logistic', learning_rate=learning_rate,
                max_depth=depth, n_estimators=estimator)
            xg_cls.fit(X_train_wv_reg, train_df_processed['recommended'])
            train_MSEs.append(roc_auc_score(
                train_df_processed['recommended'], xg_cls.predict(X_train_wv_reg)))
            val_MSEs.append(roc_auc_score(
                val_df_processed['recommended'], xg_cls.predict(X_val_wv_reg)))
            print("Training AUC: {}".format(train_MSEs[-1]))
            print("Validation AUC: {}".format(val_MSEs[-1]))
            print()

Learning Rate: 0.1, # Estimators: 1000, Depth: 1
---------------------------------------------------
Training AUC: 0.6610928933573467
Validation AUC: 0.6499316037897869

Learning Rate: 0.1, # Estimators: 1000, Depth: 2
---------------------------------------------------
Training AUC: 0.6760228864569494
Validation AUC: 0.6520758300939535

Learning Rate: 0.1, # Estimators: 2000, Depth: 1
---------------------------------------------------
Training AUC: 0.6654404824732596
Validation AUC: 0.652240628999629

Learning Rate: 0.1, # Estimators: 2000, Depth: 2
---------------------------------------------------


KeyboardInterrupt: 

In [82]:
xgb_w2v = pd.DataFrame({'learning_rate': lrs,
                        'n_estimators': estim,
                        'max_depth': ds,
                        'training_MSE': train_MSEs,
                        'validation_MSE': val_MSEs})
xgb_w2v.to_csv(RESULTS_DIR+"xgbW2V.csv", index=False)

In [83]:
xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.3, max_depth=2, n_estimators=2000)

xg_cls.fit(X_train_wv_reg, train_df_processed['recommended'])

train_MSE = roc_auc_score(
                train_df_processed['recommended'], xg_cls.predict(X_train_wv_reg))
val_MSE = roc_auc_score(
                val_df_processed['recommended'], xg_cls.predict(X_val_wv_reg))
print("Training AUC: {}".format(train_MSE))
print("Validation AUC: {}".format(val_MSE))

Training AUC: 0.688638977844757
Validation AUC: 0.6455281634152571
